# Semana 1 do challenge

## Tarefa 1 - Entender quais informações o conjunto de dados possui

### Dicionário de dados

* `customerID`: número de identificação único de cada cliente
* `Churn`: se o cliente deixou ou não a empresa 
* `gender`: gênero (masculino e feminino) 
* `SeniorCitizen`: informação sobre um cliente ter ou não idade igual ou maior que 65 anos 
* `Partner`:  se o cliente possui ou não um parceiro ou parceira
* `Dependents`: se o cliente possui ou não dependentes
* `tenure`:  meses de contrato do cliente
* `PhoneService`: assinatura de serviço telefônico 
* `MultipleLines`: assisnatura de mais de uma linha de telefone 
* `InternetService`: assinatura de um provedor internet 
* `OnlineSecurity`: assinatura adicional de segurança online 
* `OnlineBackup`: assinatura adicional de backup online 
* `DeviceProtection`: assinatura adicional de proteção no dispositivo 
* `TechSupport`: assinatura adicional de suporte técnico, menos tempo de espera
* `StreamingTV`: assinatura de TV a cabo 
* `StreamingMovies`: assinatura de streaming de filmes 
* `Contract`: tipo de contrato
* `PaperlessBilling`: se o cliente prefere receber online a fatura
* `PaymentMethod`: forma de pagamento
* `Charges.Monthly`: total de todos os serviços do cliente por mês
* `Charges.Total`: total gasto pelo cliente

### Leitura inicial dos dados

In [59]:
import pandas as pd
pd.set_option("display.max_columns", 100)

dados = pd.read_json(
    path_or_buf='Telco-Customer-Churn.json',
    orient='columns'
)
dados.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


O dataset possui informaçções gerais do cliente como ID e seu Churn. Há ainda outras quatro listas referentes a caracerísticas do cliente. 
* customer: Caractéristicas gerais do cliente
* phone: Informações sobre a conta telefônica do cliente
* internet: Informações da conta de internet do cliente
* account: Informações do contrato do cliente

### Extraindo conteúdo das colunas

O arquivo json será normalizado para um DataFrame do pandas.

In [60]:
# Extraindo dados das listas do arquivo json

customer = pd.json_normalize(data=dados.customer)
phone = pd.json_normalize(data=dados.phone)
internet = pd.json_normalize(data=dados.internet)
account = pd.json_normalize(data=dados.account, sep='_')

Concatenando os dados em um único DataFrame

In [61]:
dados = pd.concat([dados[['customerID', 'Churn']], customer, phone, internet, account], axis=1) 

## Tarefa 2 - Analisar quais os tipos de dados

Informações sobre as variáveis.

In [62]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7267 non-null   object 
 1   Churn             7267 non-null   object 
 2   gender            7267 non-null   object 
 3   SeniorCitizen     7267 non-null   int64  
 4   Partner           7267 non-null   object 
 5   Dependents        7267 non-null   object 
 6   tenure            7267 non-null   int64  
 7   PhoneService      7267 non-null   object 
 8   MultipleLines     7267 non-null   object 
 9   InternetService   7267 non-null   object 
 10  OnlineSecurity    7267 non-null   object 
 11  OnlineBackup      7267 non-null   object 
 12  DeviceProtection  7267 non-null   object 
 13  TechSupport       7267 non-null   object 
 14  StreamingTV       7267 non-null   object 
 15  StreamingMovies   7267 non-null   object 
 16  Contract          7267 non-null   object 


O dataset contém 20 variáveis. Automaticamente a coluna `ternure`foi reconhedida como do tipo inteiro e a coluna `Charges_Monthly` como do tipo float64 as demais foram reconhecidas como object. Porém, a coluna `Charges_Total` também deverá ser reconhecida como do tipo float64, esta correção é feita na próxima secção. 

A maioria das colunas são variáveis categóricas e as colunas `Charges_Monthly`, `Charges_Total` e `tenure`são numéricas.

## Tarefa 3 - Verificar quais as inconsistências nos dados

Na coluna `Charges_Total` existem células preenchidas com espaços em branco.

In [63]:
dados.query("Charges_Total == ' '").shape

(11, 21)

Já na coluna `Churn` existe incosistência semelhante porém as células são vazias.

In [64]:
dados.Churn.value_counts()

No     5174
Yes    1869
        224
Name: Churn, dtype: int64

Na coluna `SeniorCitizen` os valores estão entre 0 e 1 quando deveriam estar como Yes ou No, matendo o padrão das demais colunas.

In [65]:
dados.SeniorCitizen.value_counts()

0    6085
1    1182
Name: SeniorCitizen, dtype: int64

Verificando se há dados repetidos

In [66]:
numero_registros_unicos = dados['customerID'].nunique()
total_registros = len(dados)

print(f'Quantidade de registros = {total_registros}')
print(f'Quantidade de registros únicos = {numero_registros_unicos}')


Quantidade de registros = 7267
Quantidade de registros únicos = 7267


Portanto não há registros repetidos.

Verificando se há inconsistência quanto ao serviço telefônico. Neste caso uma inconsistência seria se um cliente que não têm registro telefônico tivesse multiplas linhas telefônicas.

In [67]:
clientes_sem_linha_telefonica = dados.query('PhoneService == "No"')
numero_clientes_sem_linha_telefonica = len(clientes_sem_linha_telefonica)

print(f'Quantidade de clientes sem linha telefônica = {numero_clientes_sem_linha_telefonica}')

print('Quantidade de clientes sem múltiplas linhas tefefônicas:')
clientes_sem_linha_telefonica['MultipleLines'].value_counts()

Quantidade de clientes sem linha telefônica = 707
Quantidade de clientes sem múltiplas linhas tefefônicas:


No phone service    707
Name: MultipleLines, dtype: int64

OK. Não há inconsistência aparente com as linhas telefônicas.

Da mesma forma será verificada a possível inconsistência relacionada ao serviço de internet. Clientes sem serviço de internet não podem ter os serviços relacionados: OnlineSecurity, OnlineBackup, DeviceProtection,
TechSuport, StreamingTV, StreamingMovies.

In [68]:
clientes_sem_servico_internet = dados.query('InternetService == "No"')
quantidade_clientes_sem_servico_internet = len(clientes_sem_servico_internet)

print(f'Quantide de clientes sem serviço de internet = {quantidade_clientes_sem_servico_internet}')
print('Número de clientes sem os serviços associados a internet:')

clientes_sem_servico_internet[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']].value_counts()


Quantide de clientes sem serviço de internet = 1581
Número de clientes sem os serviços associados a internet:


OnlineSecurity       OnlineBackup         DeviceProtection     TechSupport          StreamingTV          StreamingMovies    
No internet service  No internet service  No internet service  No internet service  No internet service  No internet service    1581
dtype: int64

Portanto também não há incosistência nesta parte.

Outra incosistência possível seria valores negativos em algumas das variáveis numéricas. A seguir é verificado isto.

In [69]:
dados.describe()

,SeniorCitizen,tenure,Charges_Monthly
count,7267.000000,7267.000000,7267.000000
mean,0.162653,32.346498,64.720098
std,0.369074,24.571773,30.129572
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.425000
50%,0.000000,29.000000,70.300000
75%,0.000000,55.000000,89.875000
max,1.000000,72.000000,118.750000


Em todas as variáveis numéricas o valor mínimo é zero portanto não há a inconsistência de números negativos. Porém, ainda será necessario ajustar a coluna "Charges_Total" que veio como string, Após a transformação dos dados de string para float será verificado novamente se há valores estranhos na coluna.

## Tarefa 4 - Corrigir as incosistências nos dados

Para os dados faltantes na coluna `Charges_Total` irei transformá-los em NaN e os demais em float.

In [70]:
import numpy as np

# Substitui espaços em branco ' ' por NaN e converte os demais casos pra float64
dados.Charges_Total = dados.Charges_Total.apply(lambda x: np.nan if x == ' ' else float(x))

Verificando se `Charges_Total` contém valores negativos.

In [71]:
dados.Charges_Total.min()

18.8

Não há valores negativos em `Charges_Total` podemos coninuar.

Verificando a quantidade de nulos em `Charges_Total`.

In [72]:
dados.Charges_Total.isna().sum()

11

Descrição dos clientes com valores nulos em `Charges_Total`

In [73]:
dados.loc[dados.Charges_Total.isna()]

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges_Monthly,Charges_Total
975,1371-DWPAZ,No,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),56.05,NaN
1775,2520-SGTTA,No,Female,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.00,NaN
1955,2775-SEFEE,No,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,NaN
2075,2923-ARZLG,No,Male,0,Yes,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,Yes,Mailed check,19.70,NaN
2232,3115-CZMZD,No,Male,0,No,Yes,0,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,20.25,NaN
2308,3213-VVOLG,No,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.35,NaN
2930,4075-WKNIU,No,Female,0,Yes,Yes,0,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Mailed check,73.35,NaN
3134,4367-NUYAO,No,Male,0,Yes,Yes,0,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Mailed check,25.75,NaN
3203,4472-LVYGI,No,Female,0,Yes,Yes,0,No,No phone service,DSL,Yes,No,Yes,Yes,Yes,No,Two year,Yes,Bank transfer (automatic),52.55,NaN
4169,5709-LVOEQ,No,Female,0,Yes,Yes,0,Yes,No,DSL,Yes,Yes,Yes,No,Yes,Yes,Two year,No,Mailed check,80.85,NaN


Os clientes com `Charges_Total` ingual a NaN têm um tempo de permanência igual a zero, ou seja, desisistiram do contrato antes de completarem um mês de contrato. Assim os valores nulos de `Charges_Total` serão substituídos pelo valor do gasto mensal `Charges_Monthly`.

In [74]:
idx_na = dados.Charges_Total.isna()
dados.loc[idx_na, 'Charges_Total'] = dados.loc[idx_na, "Charges_Monthly"]
dados.query('tenure == 0')[['Charges_Total', 'Charges_Monthly', 'tenure']]

,Charges_Total,Charges_Monthly,tenure
975,56.05,56.05,0
1775,20.00,20.00,0
1955,61.90,61.90,0
2075,19.70,19.70,0
2232,20.25,20.25,0
2308,25.35,25.35,0
2930,73.35,73.35,0
3134,25.75,25.75,0
3203,52.55,52.55,0
4169,80.85,80.85,0


De forma semelhante será feito na coluna de `Churn`.

In [75]:
# Caso haja valor vazio substitui por NaN caso contrário mantém o valor
dados.Churn = dados.Churn.apply(lambda x: np.nan if x == "" else x)

Verificando a quantidade de nulos em `Churn`.

In [76]:
dados.Churn.isna().sum()

224

Checando se existem valores vazios nas demais colunas.

In [77]:
for column in dados.columns:
    if len(dados.query(f'{column} == ""')):
        print(dados.query(f'{column} == ""'))
    if len(dados.query(f'{column} == " "')):
        print(dados.query(f'{column} == " "'))

Não há mais dados vazios.

Mudando os valores 0 e 1 da `SeniorCitizen' para Yes ou No

In [78]:
map_seniors = {0:'No', 1:'Yes'}
dados.SeniorCitizen.replace(map_seniors, inplace=True)

dados.SeniorCitizen.value_counts()

No     6085
Yes    1182
Name: SeniorCitizen, dtype: int64

Total de linhas contendo valores nulos na coluna `Churn`.

In [79]:
total_linhas_com_nulos = dados.isna().sum()
print('Total = ', total_linhas_com_nulos)

Total =  customerID            0
Churn               224
gender                0
SeniorCitizen         0
Partner               0
Dependents            0
tenure                0
PhoneService          0
MultipleLines         0
InternetService       0
OnlineSecurity        0
OnlineBackup          0
DeviceProtection      0
TechSupport           0
StreamingTV           0
StreamingMovies       0
Contract              0
PaperlessBilling      0
PaymentMethod         0
Charges_Monthly       0
Charges_Total         0
dtype: int64


Como existe uma pequena quantidade de dados nulos, quando comparamos com o tamanho da base de dados, as linhas com valores nulos na coluna `Churn` poderão ser removidas sem grades prejuízos para análises futuras.

Removendo valores nulos da base de dados.

In [80]:
numero_registros_inicial = dados.shape[0]
dados.dropna(subset=['Churn'], inplace=True)
numero_registros_final = dados.shape[0]
variacao_percentual = (numero_registros_final - numero_registros_inicial) / numero_registros_inicial * 100
print("Quantidade de registros antes da remoção dos valores nulos", numero_registros_inicial)
print("Quantidade de registros depois da remoção das linhas com valores nulos",numero_registros_final)
print(f"Porcentagem de registros removidos {variacao_percentual:.2f}%")


Quantidade de registros antes da remoção dos valores nulos 7267
Quantidade de registros depois da remoção das linhas com valores nulos 7043
Porcentagem de registros removidos -3.08%


Apenas 3,08% dos registros orignias foram perdidos no processo.

## Tarefa 5 - Traduzir os dados

Primeiro passo será substituir as ocorrências de Yes e No por Sim e Não respectivamente

In [81]:
yes_no = {'Yes': 'Sim', 'No':'Não'}

colunas_yes_no = ['Churn', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService',
                     'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 
                     'StreamingTV', 'StreamingMovies', 'PaperlessBilling']

for coluna in colunas_yes_no:
    dados[coluna].replace(yes_no, inplace=True)

Na sequência é feita a tradução dos gêneros.

In [82]:
male_female = {'Male':'Masculino', 'Female':'Feminino'}

dados.gender.replace(male_female, inplace=True)
dados.gender.value_counts()

Masculino    3555
Feminino     3488
Name: gender, dtype: int64

Tradução dos registros sem serviço de telefone. Como `No phone service` significa o mesmo que `No` será traduzido como `Não` para simplificar análises futuras.

In [83]:
dados.PhoneService.replace('No phone service', 'Não', inplace=True)
dados.MultipleLines.replace('No phone service', 'Não', inplace=True)

Tradução dos serviços de internet.

In [84]:
dados.InternetService.replace('Fiber optic', 'Fibra Óptica', inplace=True)

Tradução dos contratos.

In [85]:
contratos = {'Month-to-month' : 'Mensal', 'Two year': 'Dois anos', 'One year': 'Um ano'}
dados.Contract.replace(contratos, inplace=True)
dados.Contract.value_counts()

Mensal       3875
Dois anos    1695
Um ano       1473
Name: Contract, dtype: int64

Tradução dos métodos de pagamento.

In [86]:
metodo_pagamento = {'Electronic check': 'Cheque eletrônico', 'Mailed check':'Cheque',
                    'Bank transfer (automatic)': 'Transferência bancária',
                    'Credit card (automatic)': 'Cartão de crédito'}

dados.PaymentMethod.replace(metodo_pagamento, inplace=True)
dados.PaymentMethod.value_counts()

Cheque eletrônico         2365
Cheque                    1612
Transferência bancária    1544
Cartão de crédito         1522
Name: PaymentMethod, dtype: int64

Tradução dos registros sem serviço de internet. Da mesmo forma que nos registros sem linha telefônica `No Internet service` será traduzido como `Não` para simplificar análises futuras.

In [87]:
sem_internet = {'No internet service': 'Não'}
colunas = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
            'StreamingMovies']

for coluna in colunas:
    dados[coluna].replace(sem_internet, inplace=True)

Tradução das colunas

In [88]:
novos_nomes_colunas = ['ID_cliente', 'churn', 'genero', 'senior', 'parceiro', 'dependentes', 'tempo_permanencia',
                        'servico_telefone', 'multiplas_linhas', 'servico_internet', 'seguranca_online',
                        'backup_online', 'protecao_dispositivo', 'suporte_tecnico', 'streaming_TV',
                        'streaming_filmes', 'contrato', 'fatura_online', 'metodo_pagamento',
                        'gastos_mensais', 'gastos_totais']

dados.columns = novos_nomes_colunas
dados.head()

,ID_cliente,churn,genero,senior,parceiro,dependentes,tempo_permanencia,servico_telefone,multiplas_linhas,servico_internet,seguranca_online,backup_online,protecao_dispositivo,suporte_tecnico,streaming_TV,streaming_filmes,contrato,fatura_online,metodo_pagamento,gastos_mensais,gastos_totais
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,Não,Sim,Não,Sim,Sim,Não,Um ano,Sim,Cheque,65.6,593.30
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,Não,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,59.9,542.40
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,Não,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,73.9,280.85
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,Não,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,98.0,1237.85
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,Não,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,83.9,267.40


## Tarefa 6 - Criar coluna de contas diárias

In [89]:
dados['gastos_diarios'] = dados.gastos_mensais / 30

In [90]:
dados.head()

,ID_cliente,churn,genero,senior,parceiro,dependentes,tempo_permanencia,servico_telefone,multiplas_linhas,servico_internet,seguranca_online,backup_online,protecao_dispositivo,suporte_tecnico,streaming_TV,streaming_filmes,contrato,fatura_online,metodo_pagamento,gastos_mensais,gastos_totais,gastos_diarios
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,Não,Sim,Não,Sim,Sim,Não,Um ano,Sim,Cheque,65.6,593.30,2.186667
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,Não,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,59.9,542.40,1.996667
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,Não,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,73.9,280.85,2.463333
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,Não,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,98.0,1237.85,3.266667
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,Não,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,83.9,267.40,2.796667


Alterando posição da coluna de média de gastos diária.

In [91]:
dados.insert(19, 'gastos_diarios', dados.pop('gastos_diarios'))
dados.head()

,ID_cliente,churn,genero,senior,parceiro,dependentes,tempo_permanencia,servico_telefone,multiplas_linhas,servico_internet,seguranca_online,backup_online,protecao_dispositivo,suporte_tecnico,streaming_TV,streaming_filmes,contrato,fatura_online,metodo_pagamento,gastos_diarios,gastos_mensais,gastos_totais
0,0002-ORFBO,Não,Feminino,Não,Sim,Sim,9,Sim,Não,DSL,Não,Sim,Não,Sim,Sim,Não,Um ano,Sim,Cheque,2.186667,65.6,593.30
1,0003-MKNFE,Não,Masculino,Não,Não,Não,9,Sim,Sim,DSL,Não,Não,Não,Não,Não,Sim,Mensal,Não,Cheque,1.996667,59.9,542.40
2,0004-TLHLJ,Sim,Masculino,Não,Não,Não,4,Sim,Não,Fibra Óptica,Não,Não,Sim,Não,Não,Não,Mensal,Sim,Cheque eletrônico,2.463333,73.9,280.85
3,0011-IGKFF,Sim,Masculino,Sim,Sim,Não,13,Sim,Não,Fibra Óptica,Não,Sim,Sim,Não,Sim,Sim,Mensal,Sim,Cheque eletrônico,3.266667,98.0,1237.85
4,0013-EXCHZ,Sim,Feminino,Sim,Sim,Não,3,Sim,Não,Fibra Óptica,Não,Não,Não,Sim,Sim,Não,Mensal,Sim,Cheque,2.796667,83.9,267.40


## Observações finais.

A tabelas com variáveis categóricas textuais é ótima para a interpretação humana. Porém para os processamentos de máquina textos não são fáceis de processar. Assim todas as variáveis categóricas textuais, do tipo binárias (sim ou não, feminino ou masculino) serão convertidas em variáveis categóricas binárias (0 ou 1).

Criação do mapa de conversões

In [92]:
mapa_binarios = {
    'Sim' : 1,
    'Não' : 0,
    'Feminino' : 1,
    'Masculino' : 0,
}
colunas_binarias = ['churn', 'genero', 'senior', 'parceiro', 'dependentes',
                    'servico_telefone', 'multiplas_linhas', 'seguranca_online',
                    'backup_online', 'protecao_dispositivo', 'suporte_tecnico',
                    'streaming_TV', 'streaming_filmes', 'fatura_online', ]

dados[colunas_binarias] = dados[colunas_binarias].replace(mapa_binarios)
dados.head()


,ID_cliente,churn,genero,senior,parceiro,dependentes,tempo_permanencia,servico_telefone,multiplas_linhas,servico_internet,seguranca_online,backup_online,protecao_dispositivo,suporte_tecnico,streaming_TV,streaming_filmes,contrato,fatura_online,metodo_pagamento,gastos_diarios,gastos_mensais,gastos_totais
0,0002-ORFBO,0,1,0,1,1,9,1,0,DSL,0,1,0,1,1,0,Um ano,1,Cheque,2.186667,65.6,593.30
1,0003-MKNFE,0,0,0,0,0,9,1,1,DSL,0,0,0,0,0,1,Mensal,0,Cheque,1.996667,59.9,542.40
2,0004-TLHLJ,1,0,0,0,0,4,1,0,Fibra Óptica,0,0,1,0,0,0,Mensal,1,Cheque eletrônico,2.463333,73.9,280.85
3,0011-IGKFF,1,0,1,1,0,13,1,0,Fibra Óptica,0,1,1,0,1,1,Mensal,1,Cheque eletrônico,3.266667,98.0,1237.85
4,0013-EXCHZ,1,1,1,1,0,3,1,0,Fibra Óptica,0,0,0,1,1,0,Mensal,1,Cheque,2.796667,83.9,267.40


Exportando arquivo csv

In [93]:
dados.to_csv('dados_clientes_alura_voz.csv', index=False)